In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import torch

import time


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import time

In [2]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')



In [3]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./Defense_Distillation.csv", mode='a', index=False)




In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
epochs = 50
training_batch_size = 128
validation_batch_size = 128
learning_rate = 0.01
baseline_temperature = 1
distilled_temperature = 100


x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)


input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize models
teacher_model = DNNModel(input_shape, output_shape).to(device)
student_model = DNNModel(input_shape, output_shape).to(device)


In [5]:
# Training function for the teacher model
def train_teacher_model(model, train_loader, epochs, temperature):
    min_delta = 0.001
    patience = 5
    patience_counter = 0
    best_loss = float('100000000')

    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_loss = 0.0
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs / temperature, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        avg_train_loss = train_loss / len(train_loader)
        
        model.eval()
        val_train_loss = 0.0
        correct_predictions = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()
    
        avg_val_loss = val_train_loss / len(val_loader)
        val_accuracy = correct_predictions / len(val_dataset)
    
        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
        # Early stopping check using min_delta
        if best_loss - avg_val_loss > min_delta:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
    
        if patience_counter >= patience:
            print("Early stopping triggered")
            break
    torch.save(model.state_dict(), 'teacher_model.pth')



In [6]:
# Train the teacher model
# train_teacher_model(teacher_model, train_loader, epochs, baseline_temperature)

In [7]:
teacher_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Defense_Distillation/teacher_model.pth"))

/tmp/ipykernel_3618/2105613291.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_De

<All keys matched successfully>

In [8]:
# Distillation loss function
def distillation_loss(student_outputs, teacher_outputs, temperature, alpha):
    soft_targets = F.softmax(teacher_outputs / temperature, dim=1)
    student_log_probs = F.log_softmax(student_outputs / temperature, dim=1)
    distillation_loss = nn.KLDivLoss()(student_log_probs, soft_targets) * (temperature ** 2)
    return distillation_loss

# Training function for the student model using distillation
def train_student_model(student_model, teacher_model, train_loader, epochs, temperature, alpha=0.5):
    min_delta = 0.001
    patience = 5
    patience_counter = 0
    best_loss = float('100000000')
    
    student_model.train()
    teacher_model.eval()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(student_model.parameters(), lr=learning_rate)
    
    train_loss = 0.0

    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            teacher_outputs = teacher_model(inputs).detach()
            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, temperature, alpha) + criterion(student_outputs, labels) * (1. - alpha)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        avg_train_loss = train_loss / len(train_loader)
        
        student_model.eval()
        val_train_loss = 0.0
        correct_predictions = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                teacher_outputs = teacher_model(inputs).detach()
                student_outputs = student_model(inputs)
                loss = distillation_loss(student_outputs, teacher_outputs, temperature, alpha) + criterion(student_outputs, labels) * (1. - alpha)
                val_train_loss += loss.item()
                _, predicted = torch.max(student_outputs.data, 1)
                correct_predictions += (predicted == labels).sum().item()
    
        avg_val_loss = val_train_loss / len(val_loader)
        val_accuracy = correct_predictions / len(val_dataset)
    
        print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
        # Early stopping check using min_delta
        if best_loss - avg_val_loss > min_delta:
            best_loss = avg_val_loss
            patience_counter = 0
        else:
            patience_counter += 1
    
        if patience_counter >= patience:
            print("Early stopping triggered")
            break
    torch.save(student_model.state_dict(), 'student_model.pth')



In [9]:
# Train the student model
# train_student_model(student_model, teacher_model, train_loader, epochs, distilled_temperature)


In [10]:
student_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Defense_Distillation/student_model.pth"))

/tmp/ipykernel_3618/3271045883.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student_model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_De

<All keys matched successfully>

In [11]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./Defense_Distillation.csv", mode='a', index=False, header=False)




In [12]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def8/y_pred_{attack_name}{eps}_Def8.npy", all_preds)


In [13]:

# calculate_performance_metrics(x_test, y_test, student_model, 'DNN', 'Baseline', '0')

In [14]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./Defense_Distillation.csv", mode='a', index=False, header=False)

In [15]:
# percentage = ["100", "50", "20", "1"]
# model_name = ["XGB", "RF", "DT" ]

percentage = ["10"]
model_name = ["RF"]

In [16]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(112, 56) (112,)
Accuracy: 0.23214285714285715

macro
Precision: 0.1826923076923077
Recall: 0.396875
F1 Score: 0.19518716577540107

weighted
Precision: 0.157967032967033
Recall: 0.23214285714285715
F1 Score: 0.12127578304048893

start RF 10 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_BIM_Def8.npy not found
start RF 10 FGSM
(6, 56) (6,)
Accuracy: 0.3333333333333333

macro
Precision: 0.5
Recall: 0.16666666666666666
F1 Score: 0.25

weighted
Precision: 1.0
Recall: 0.3333333333333333
F1 Score: 0.5

start RF 10 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input10/x_test_adv_PGD_Def8.npy not found
start RF 10 DF
(15279, 56) (15279,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.16401596963152038

macro
Precision: 0.4748548881474866
Recall: 0.34255580537322367
F1 Score: 0.14719230178044923

weighted
Precision: 0.9151321144417129
Recall: 0.16401596963152038
F1 Score: 0.2616418483591014

start RF 10 AutoPGD
(252, 56) (252,)
Accuracy: 0.011904761904761904

macro
Precision: 0.1
Recall: 0.00625
F1 Score: 0.011764705882352941

weighted
Precision: 0.19047619047619047
Recall: 0.011904761904761904
F1 Score: 0.02240896358543417

start RF 10 ZOO
(336, 56) (336,)
Accuracy: 0.23214285714285715

macro
Precision: 0.1826923076923077
Recall: 0.396875
F1 Score: 0.19518716577540107

weighted
Precision: 0.157967032967033
Recall: 0.23214285714285715
F1 Score: 0.12127578304048893

start RF 10 CaFA
(69, 56) (69,)
Accuracy: 0.9130434782608695

macro
Precision: 0.875725338491296
Recall: 0.923202614379085
F1 Score: 0.8943877551020408

weighted
Precision: 0.9249852829871331
Recall: 0.9130434782608695
F1 Score: 0.9156166814551907

start RF 10 SINIFGSM
/home/jovyan/Sample_Base

In [17]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(109, 56) (109,)
Accuracy: 0.25688073394495414

macro
Precision: 0.3321123321123321
Recall: 0.38396624472573837
F1 Score: 0.2477634830024708

weighted
Precision: 0.38261025416988714
Recall: 0.25688073394495414
F1 Score: 0.21053470832066398

start RF 10 BIM
(20, 56) (20,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 FGSM
(79, 56) (79,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 PGD
(20, 56) (20,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 DF
(15690, 56) (15690,)
Accuracy: 0.17782026768642448

macro
Precision: 0.46682704467678576
Recall: 0.28490843977136404
F1 Score: 0.1564730016598229

weighted
Precision: 0.9007145036048739
Recall: 0.17782026768642448
F1 Score: 0.2843342304649887

start RF 10 AutoPGD
(264, 56) (264,)
Accuracy: 0.011363636363636364

macro
Precision: 0.029411764705882353
Recall: 0.006944444444444444
F1 Score: 0.011235955056179775

weighted
Precision: 0.0481283422459893
Recall: 0.011363636363636364
F1 Score: 0.01838610827374872

start RF 10 ZOO
(327, 56) (327,)
Accuracy: 0.25688073394495414

macro
Precision: 0.3321123321123321
Recall: 0.38396624472573837
F1 Score: 0.2477634830024708

weighted
Precision: 0.3826102541698872
Recall: 0.25688073394495414
F1 Score: 0.21053470832066398

start RF 10 CaFA
(49, 56) (49,)
Accuracy: 0.9795918367346939

macro
Precision: 0.5
Recall: 0.48979591836

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0



/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

start RF 10 baseline
(57, 56) (57,)
Accuracy: 0.08771929824561403

macro
Precision: 0.22727272727272727
Recall: 0.049019607843137254
F1 Score: 0.08064516129032258

weighted
Precision: 0.4066985645933014
Recall: 0.08771929824561403
F1 Score: 0.14431239388794564

start RF 10 BIM
(2, 56) (2,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude2Attack/UNSW_Input10/x_test_adv_FGSM_Def8.npy not found
start RF 10 PGD
(2, 56) (2,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 DF
(15356, 56) (15356,)
Accuracy: 0.16859859338369368

macro
Precision: 0.47076496441008264
Recall: 0.30412825423098144
F1 Score: 0.1496378293077598

weighted
Precision: 0.9100513609190581
Recall: 0.16859859338369368
F1 Score: 0.2711256382059807

start RF 10 AutoPGD
(170, 56) (170,)
Acc

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.3625

macro
Precision: 0.5
Recall: 0.18125
F1 Score: 0.26605504587155965

weighted
Precision: 1.0
Recall: 0.3625
F1 Score: 0.5321100917431193

start RF 10 FGSM
(394, 56) (394,)
Accuracy: 0.005076142131979695

macro
Precision: 0.5012722646310432
Recall: 0.5012722646310432
F1 Score: 0.005076142131979695

weighted
Precision: 0.9974683871301068
Recall: 0.005076142131979695
F1 Score: 0.005076142131979695

start RF 10 PGD
(80, 56) (80,)
Accuracy: 0.3625

macro
Precision: 0.5
Recall: 0.18125
F1 Score: 0.26605504587155965

weighted
Precision: 1.0
Recall: 0.3625
F1 Score: 0.5321100917431193

start RF 10 DF
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude3Attack/UNSW_Input10/x_test_adv_DF_Def8.npy not found
start RF 10 AutoPGD
(130, 56) (130,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.1

macro
Precision: 0.19117647058823528
Recall: 0.05963302752293578
F1 Score: 0.09090909090909091

weighted
Precision: 0.3205882352941176
Recall: 0.1
F1 Score: 0.15244755244755245

start RF 10 ZOO
(156, 56) (156,)
Accuracy: 0.25

macro
Precision: 0.33496732026143794
Recall: 0.23901808785529716
F1 Score: 0.22683949675943577

weighted
Precision: 0.5155228758169935
Recall: 0.25
F1 Score: 0.3143347312237895

start RF 10 CaFA
(5, 56) (5,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 SINIFGSM
(5, 56) (5,)
Accuracy: 0.6

macro
Precision: 0.3
Recall: 0.5
F1 Score: 0.375

weighted
Precision: 0.36
Recall: 0.6
F1 Score: 0.45

start RF 10 VNIFGSM
(6, 56) (6,)
Accuracy: 0.5

macro
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333

weighted
Precision: 0.25
Recall: 0.5
F1 Score: 0.3333333333333333



/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

In [15]:
epsilon = 0

Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.04075235109717868

macro
Precision: 0.232475715410869
Recall: 0.05307017543859649
F1 Score: 0.03961038961038961

weighted
Precision: 0.4399965104919795
Recall: 0.04075235109717868
F1 Score: 0.06961283230875707

start BIM
Accuracy: 0.1725254394079556

macro
Precision: 0.2182083178629896
Recall: 0.19804739724845818
F1 Score: 0.17158097759329524

weighted
Precision: 0.2718693066702186
Recall: 0.1725254394079556
F1 Score: 0.1842859296754388

start FGSM
Accuracy: 0.2641509433962264

macro
Precision: 0.24895412844036696
Recall: 0.28113562488802435
F1 Score: 0.25083240843507215

weighted
Precision: 0.24467841440193874
Recall: 0.2641509433962264
F1 Score: 0.24040374426737585

start PGD
Accuracy: 0.1725254394079556

macro
Precision: 0.2182083178629896
Recall: 0.19804739724845818
F1 Score: 0.17158097759329524

weighted
Precision: 0.2718693066702186
Recall: 0.1725254394079556
F1 Score: 0.1842859296754388

start DF
Accuracy: 0.18365119783817493

macro
Precision: 0.458666

In [19]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(41, 56) (41,)
Accuracy: 0.2682926829268293

macro
Precision: 0.25
Recall: 0.18333333333333332
F1 Score: 0.21153846153846154

weighted
Precision: 0.36585365853658536
Recall: 0.2682926829268293
F1 Score: 0.3095684803001876

start RF 10 BIM
(80, 56) (80,)
Accuracy: 0.1875

macro
Precision: 0.5547945205479452
Recall: 0.5486111111111112
F1 Score: 0.1873730270354743

weighted
Precision: 0.9109589041095891
Recall: 0.1875
F1 Score: 0.17924675730582904

start RF 10 FGSM
(50, 56) (50,)
Accuracy: 0.4

macro
Precision: 0.6052631578947368
Recall: 0.6428571428571428
F1 Score: 0.3961352657004831

weighted
Precision: 0.8736842105263157
Recall: 0.4
F1 Score: 0.42898550724637674

start RF 10 PGD
(80, 56) (80,)
Accuracy: 0.1875

macro
Precision: 0.5547945205479452
Recall: 0.5486111111111112
F1 Score: 0.1873730270354743

weighted
Precision: 0.9109589041095891
Recall: 0.1875
F1 Score: 0.17924675730582904

start RF 10 DF
(23577, 56) (23577,)
Accuracy: 0.08639776052932943

macro
Precisi

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
epsilon = 0
Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(186, 56) (186,)
Accuracy: 0.03225806451612903

macro
Precision: 0.5054945054945055
Recall: 0.5108695652173914
F1 Score: 0.03214616096207215

weighted
Precision: 0.989365473236441
Recall: 0.03225806451612903
F1 Score: 0.04232938438124795

start RF 10 BIM
(1, 56) (1,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 FGSM
(1, 56) (1,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 PGD
(1, 56) (1,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 DF
(883, 56) (883,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.48471121177802945

macro
Precision: 0.4967298115919708
Recall: 0.47793251576248874
F1 Score: 0.35498087089984637

weighted
Precision: 0.9226062832598944
Recall: 0.48471121177802945
F1 Score: 0.6219763956870161

start RF 10 AutoPGD
(552, 56) (552,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.021739130434782608

macro
Precision: 0.5
Recall: 0.010869565217391304
F1 Score: 0.02127659574468085

weighted
Precision: 1.0
Recall: 0.021739130434782608
F1 Score: 0.0425531914893617

start RF 10 ZOO
(558, 56) (558,)
Accuracy: 0.03225806451612903

macro
Precision: 0.5054945054945055
Recall: 0.5108695652173914
F1 Score: 0.03214616096207215

weighted
Precision: 0.9893654732364409
Recall: 0.03225806451612903
F1 Score: 0.04232938438124795

start RF 10 CaFA
(36, 56) (36,)
Accuracy: 0.9722222222222222

macro
Precision: 0.9848484848484849
Recall: 0.875
F1 Score: 0.9208791208791209

weighted
Precision: 0.9730639730639731
Recall: 0.9722222222222222
F1 Score: 0.9704517704517706

start RF 10 SINIFGSM
(5, 56) (5,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 10 VNIFGSM
(2, 56) (2,)
Accuracy: 0.5

macro
Precision: 0.5
Recall: 0.25
F1 Score: 0.3333333333333333

weighted
Precision: 1.0
Recall: 0.5
F1 Score: 0.66

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

In [21]:
epsilon = 0

Def = "Def8"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]


active_learning_name = ["DensityWeighted", "BatchMode"]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student_model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start RF 10 baseline
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_DensityWeighted/UNSW_Input10/x_test_adv_baseline_Def8.npy not found
start RF 10 BIM
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 FGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.3469387755102041

macro
Precision: 0.5
Recall: 0.17346938775510204
F1 Score: 0.25757575757575757

weighted
Precision: 1.0
Recall: 0.3469387755102041
F1 Score: 0.5151515151515151

start RF 10 PGD
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 DF
Accuracy: 0.16120728814414778

macro
Precision: 0.307592117702931
Recall: 0.10112884751951783
F1 Score: 0.14029675796754443

weighted
Precision: 0.5488147967924072
Recall: 0.16120728814414778
F1 Score: 0.2466697753934434

start RF 10 AutoPGD
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 10 ZOO
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_DensityWeighted/UNSW_Input10/x_test_adv_ZOO_Def8.npy not found
start RF 10 CaFA
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_DensityWeighted/UNSW_Input10/x_test_

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.10683760683760683

macro
Precision: 0.3424107142857143
Recall: 0.42726148773954253
F1 Score: 0.10407239819004525

weighted
Precision: 0.54936278998779
Recall: 0.10683760683760683
F1 Score: 0.06408322697915458

start RF 10 BIM
Accuracy: 0.4125

macro
Precision: 0.44594594594594594
Recall: 0.21710526315789475
F1 Score: 0.2920353982300885

weighted
Precision: 0.8472972972972974
Recall: 0.4125
F1 Score: 0.5548672566371682

start RF 10 FGSM
Accuracy: 0.6163988463123198

macro
Precision: 0.49656205822105537
Recall: 0.34811356828755335
F1 Score: 0.38235209700261846

weighted
Precision: 0.9867292272280838
Recall: 0.6163988463123198
F1 Score: 0.7584874685505387

start RF 10 PGD
Accuracy: 0.4125

macro
Precision: 0.44594594594594594
Recall: 0.21710526315789475
F1 Score: 0.2920353982300885

weighted
Precision: 0.8472972972972974
Recall: 0.4125
F1 Score: 0.5548672566371682

start RF 10 DF
Accuracy: 0.2190486984774642

macro
Precision: 0.4798110077879286
Recall: 0.35184464676570915
F1 S

In [15]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, student_model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")